# Basic Prompt Structures Tutorial

## Overview

This tutorial explores two essential prompt structures used in AI interactions:
1. Single-turn prompts
2. Multi-turn prompts (conversations)

We'll utilize Amazon Nova via OpenRouter and LangChain to illustrate these concepts.

## Motivation

Grasping various prompt structures is essential for effective AI communication. Single-turn prompts excel in quick, direct queries, while multi-turn prompts facilitate more nuanced, context-rich exchanges. Proficiency in these structures enhances the versatility and efficacy of AI applications across diverse scenarios.

## Key Components

1. **Single-turn Prompts**: One-time interactions with the AI model.
2. **Multi-turn Prompts**: Sequential exchanges that preserve context.
3. **Prompt Templates**: Standardized structures for consistent AI querying.
4. **Conversation Chains**: Techniques for maintaining context across multiple interactions.

## Setup

First, let's import the necessary libraries and set up our environment.

In [1]:
from os import getenv
from typing import List

from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_openai.chat_models import ChatOpenAI

load_dotenv()


# Initialize the language model
llm = ChatOpenAI(
    openai_api_key=getenv("OPENROUTER_API_KEY"),
    openai_api_base=getenv("OPENROUTER_BASE_URL"),
    model_name="amazon/nova-lite-v1",
)

## 1. Single-turn Prompts

Single-turn prompts are one-shot interactions with the language model. They consist of a single input (prompt) and generate a single output (response).

In [2]:
single_turn_prompt = "What are the three primary colors?"
print(llm.invoke(single_turn_prompt).content)

The three primary colors are:

1. **Red**
2. **Blue**
3. **Yellow**

These colors are fundamental in color theory and can be combined to create a wide range of other colors.


Now, let's use a PromptTemplate to create a more structured single-turn prompt:

In [3]:
structured_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Provide a brief explanation of {topic} and list its three main components.",
)

chain = structured_prompt | llm
print(chain.invoke({"topic": "color theory"}).content)

**Color Theory Explanation:**

Color theory is a framework that explains how colors interact and the effects they produce. It is essential in various fields such as art, design, and marketing. The theory helps artists and designers understand how to create harmony, contrast, and emotional responses through color.

**Three Main Components of Color Theory:**

1. **Color Wheel:**
   - A visual representation of colors arranged in a circular format.
   - It shows the relationships between primary, secondary, and tertiary colors.
   - The color wheel is divided into primary (red, blue, yellow), secondary (green, orange, purple), and tertiary colors (mixtures of primary and secondary colors).

2. **Color Harmony:**
   - Refers to the pleasing arrangement of colors.
   - Common schemes include complementary (colors opposite each other on the wheel), analogous (colors next to each other), and triadic (three colors evenly spaced on the wheel).

3. **Color Properties:**
   - **Hue:** The actual 

You can learn more about LangChain Expression Language from [LCEL](https://python.langchain.com/docs/concepts/lcel/)

## 2. Multi-turn Prompts (Conversations)

Multi-turn prompts involve a series of interactions with the language model, allowing for more complex and context-aware conversations.

In [4]:
class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """In memory implementation of chat message history."""

    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """Add a list of messages to the store"""
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []


# Here we use a global variable to store the chat message history.
# This will make it easier to inspect it to see the underlying results.
store = {}


def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

In [5]:
prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = prompt | llm

conversation = RunnableWithMessageHistory(
    chain,
    get_by_session_id,
    input_messages_key="question",
    history_messages_key="history",
)

print(
    conversation.invoke(
        input={
            "question": "Hi, I'm learning about space. Can you tell me about planets?"
        },
        config={"configurable": {"session_id": "foo"}},
    ).content
)
print(
    conversation.invoke(
        input={"question": "What's the largest planet in our solar system?"},
        config={"configurable": {"session_id": "foo"}},
    ).content
)
print(
    conversation.invoke(
        input={"question": "How does its size compare to Earth?"},
        config={"configurable": {"session_id": "foo"}},
    ).content
)

Absolutely! Planets are celestial bodies that orbit a star, such as the Sun in our solar system. Here are some key points about planets:

### Types of Planets
1. **Terrestrial Planets (Inner Planets)**
   - **Mercury**: The closest planet to the Sun, it has a rocky surface and no atmosphere.
   - **Venus**: Similar in size to Earth, it has a thick, toxic atmosphere and is the hottest planet due to a runaway greenhouse effect.
   - **Earth**: The only known planet to support life, it has a diverse atmosphere and liquid water.
   - **Mars**: Known as the "Red Planet," it has a thin atmosphere, and there is evidence of past water flow.

2. **Gas Giants (Outer Planets)**
   - **Jupiter**: The largest planet in the solar system, it has a massive atmosphere composed mostly of hydrogen and helium, and it has many moons.
   - **Saturn**: Famous for its extensive ring system, it is also a gas giant with a composition similar to Jupiter.
   - **Uranus**: It has a blue-green color due to methane 

Let's compare how single-turn and multi-turn prompts handle a series of related questions:

In [6]:
# Single-turn prompts
prompts = [
    "What is the capital of France?",
    "What is its population?",
    "What is the city's most famous landmark?",
]

print("Single-turn responses:")
for prompt in prompts:
    print(f"Q: {prompt}")
    print(f"A: {llm.invoke(prompt).content}\n")

# Multi-turn prompts
print("Multi-turn responses:")
for prompt in prompts:
    print(f"Q: {prompt}")
    response = conversation.invoke(
        input={"question": prompt}, config={"configurable": {"session_id": "bar"}}
    ).content
    print(f"A: {response}\n")

Single-turn responses:
Q: What is the capital of France?
A: The capital of France is Paris. Paris is not only the capital but also the largest city in France, known for its rich history, culture, and significant contributions to art, fashion, and cuisine. It is home to iconic landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral. Paris is a major global city and a popular tourist destination, attracting millions of visitors each year.

Q: What is its population?
A: To provide an accurate answer regarding the population of a specific location, I need more context about which place you are referring to. Populations can vary greatly depending on whether you are talking about a country, city, town, or any other geographic area. Here are a few examples:

1. **Country**: 
   - **United States**: As of 2023, the population is approximately 333 million.
   - **China**: As of 2023, the population is approximately 1.4 billion.
   - **India**: As of 2023, the population 